In [ ]:
!pip install -q sklearn
%tensorflow_version 2.x 
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf


In [ ]:
x_train = pd.read_csv('/content/train.csv') # training data
x_test = pd.read_csv('/content/test.csv') # testing data



NameError: ignored

In [ ]:
y_train = x_train.pop('Daily Summary')
data = pd.read_csv('/content/train.csv') # training data
dummies = pd.get_dummies(data['Daily Summary'],dummy_na=True)
dummies.head()
df2 = pd.concat([y_train, dummies], axis=1)
df2.head()

,Daily Summary,Breezy and foggy starting in the evening.,Breezy and foggy until morning.,Breezy and mostly cloudy overnight.,Breezy and partly cloudy in the afternoon.,Breezy in the morning and foggy in the evening.,Breezy in the morning and mostly cloudy starting in the evening.,Breezy in the morning and mostly cloudy starting in the morning.,Breezy in the morning and partly cloudy starting in the evening continuing until night.,Breezy overnight and mostly cloudy throughout the day.,Breezy overnight and overcast throughout the day.,Breezy overnight and partly cloudy throughout the day.,Breezy overnight and partly cloudy until evening.,Breezy starting in the afternoon continuing until evening and foggy in the evening.,Breezy starting in the afternoon continuing until evening and foggy starting in the evening.,Breezy starting in the afternoon continuing until night and mostly cloudy starting in the evening.,Breezy starting in the morning continuing until afternoon and mostly cloudy starting in the morning.,Breezy starting in the morning continuing until afternoon and partly cloudy starting in the morning.,Breezy starting in the morning continuing until night.,Breezy starting overnight continuing until afternoon and foggy starting in the morning continuing until evening.,Breezy starting overnight continuing until afternoon and mostly cloudy starting overnight continuing until evening.,Breezy starting overnight continuing until morning and foggy in the evening.,Breezy starting overnight continuing until morning and foggy overnight.,Breezy starting overnight continuing until morning and partly cloudy starting in the morning.,Breezy starting overnight continuing until morning and partly cloudy starting overnight continuing until evening.,Breezy starting overnight continuing until morning.,Breezy until afternoon and mostly cloudy throughout the day.,Breezy until afternoon and overcast throughout the day.,Breezy until evening and foggy in the morning.,Breezy until morning and mostly cloudy throughout the day.,Clear throughout the day.,Foggy in the afternoon.,Foggy in the evening.,Foggy in the morning and breezy starting in the afternoon continuing until night.,Foggy in the morning.,Foggy overnight and breezy in the morning.,Foggy overnight.,Foggy starting in the afternoon and breezy starting in the afternoon continuing until evening.,Foggy starting in the afternoon continuing until evening.,Foggy starting in the afternoon continuing until night.,...,Partly cloudy starting overnight and breezy starting in the evening.,Partly cloudy starting overnight and breezy starting in the morning continuing until afternoon.,Partly cloudy starting overnight continuing until afternoon and breezy in the afternoon.,Partly cloudy starting overnight continuing until afternoon.,Partly cloudy starting overnight continuing until evening and breezy in the evening.,Partly cloudy starting overnight continuing until evening and breezy in the morning.,Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until afternoon.,Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until evening.,Partly cloudy starting overnight continuing until evening and windy starting in the morning continuing until evening.,Partly cloudy starting overnight continuing until evening.,Partly cloudy starting overnight continuing until morning and breezy starting in the morning continuing until afternoon.,Partly cloudy starting overnight continuing until morning.,Partly cloudy starting overnight continuing until night and breezy in the afternoon.,Partly cloudy starting overnight continuing until night and breezy in the morning.,Partly cloudy starting overnight continuing until night and windy starting in the morning continuing until afternoon.,Partly cloudy starting overnight continuing until night.,Partly cloudy starting overnight.,Partly cloudy throughout the day and breezy in the aft

In [ ]:
y_test = x_test.pop('Daily Summary')
data1 = pd.read_csv('/content/test.csv') # testing data
dummies = pd.get_dummies(data1['Daily Summary'],dummy_na=True)
dummies.head()
df3 = pd.concat([y_test, dummies], axis=1)
df3.head()   

,Daily Summary,Clear throughout the day.,Drizzle starting in the evening.,Drizzle until morning.,Foggy in the morning.,Foggy overnight.,Foggy starting in the evening.,Foggy starting overnight continuing until morning.,Foggy until morning.,Light rain overnight.,Light rain starting overnight.,Mostly cloudy starting in the afternoon.,Mostly cloudy starting in the morning.,Mostly cloudy starting overnight.,Mostly cloudy throughout the day.,Mostly cloudy until evening.,Mostly cloudy until morning.,Mostly cloudy until night.,Overcast throughout the day.,Partly cloudy overnight.,Partly cloudy starting in the afternoon continuing until evening.,Partly cloudy starting in the afternoon.,Partly cloudy starting in the morning continuing until afternoon.,Partly cloudy starting in the morning continuing until evening.,Partly cloudy starting in the morning continuing until night.,Partly cloudy starting in the morning.,Partly cloudy starting overnight.,Partly cloudy throughout the day.,Partly cloudy until evening.,Partly cloudy until night.,Rain throughout the day.,Rain until afternoon.,Rain until morning.,NaN
0,Partly cloudy starting in the afternoon.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Partly cloudy starting in the afternoon.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Partly cloudy starting in the afternoon.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Partly cloudy starting in the afternoon.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Partly cloudy starting in the afternoon.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
x_train.rename(columns={'Precip Type': 'precipitation'}, inplace=True)
x_test.rename(columns={'Precip Type': 'precipitation'}, inplace=True)
skiprows=1

In [ ]:
x_train.head(5)

,Formatted Date,Summary,precipitation,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Cloud Cover,Pressure (millibars)
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251,15.8263,0,1015.13
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259,15.8263,0,1015.63
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204,14.9569,0,1015.94
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269,15.8263,0,1016.41
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259,15.8263,0,1016.51


In [ ]:
x_train[x_train.isnull().any(axis=1)]
x_train.fillna('', inplace=True)

In [ ]:
x_train.isnull().sum()

Formatted Date              0
Summary                     0
precipitation               0
Temperature (C)             0
Apparent Temperature (C)    0
Humidity                    0
Wind Speed (km/h)           0
Wind Bearing (degrees)      0
Visibility (km)             0
Cloud Cover                 0
Pressure (millibars)        0
dtype: int64

In [ ]:
print(y_train.head())
print(y_test.head())

0    Partly cloudy throughout the day.
1    Partly cloudy throughout the day.
2    Partly cloudy throughout the day.
3    Partly cloudy throughout the day.
4    Partly cloudy throughout the day.
Name: Daily Summary, dtype: object
0    Partly cloudy starting in the afternoon.
1    Partly cloudy starting in the afternoon.
2    Partly cloudy starting in the afternoon.
3    Partly cloudy starting in the afternoon.
4    Partly cloudy starting in the afternoon.
Name: Daily Summary, dtype: object


In [ ]:
x_train.describe()

,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Cloud Cover,Pressure (millibars)
count,94268.000000,94268.000000,94268.000000,94268.000000,94268.000000,94268.000000,94268.0,94268.000000
mean,11.949454,10.872365,0.733875,10.838893,187.707907,10.355367,0.0,1002.864099
std,9.601178,10.752540,0.196034,6.950710,107.652333,4.172897,0.0,118.290080
min,-21.822222,-27.716667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,4.527778,2.244444,0.600000,5.796000,114.000000,8.372000,0.0,1011.800000
50%,12.077778,12.077778,0.780000,10.014200,180.000000,10.046400,0.0,1016.330000
75%,18.866667,18.866667,0.890000,14.151900,290.000000,14.812000,0.0,1021.090000
max,39.905556,39.344444,1.000000,63.852600,359.000000,16.100000,0.0,1046.380000


In [ ]:
CATEGORICAL_COLUMNS = ['Summary', 'precipitation']
NUMERIC_COLUMNS = ['Temperature (C)', 'Apparent Temperature (C)','Humidity','Wind Speed (km/h)','Wind Bearing (degrees)','Visibility (km)','Pressure (millibars)']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = x_train[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='Summary', vocabulary_list=('Partly Cloudy', 'Mostly Cloudy', 'Overcast', 'Foggy', 'Breezy and Mostly Cloudy', 'Clear', 'Breezy and Partly Cloudy', 'Breezy and Overcast', 'Humid and Mostly Cloudy', 'Humid and Partly Cloudy', 'Windy and Foggy', 'Windy and Overcast', 'Breezy and Foggy', 'Windy and Partly Cloudy', 'Breezy', 'Dry and Partly Cloudy', 'Windy and Mostly Cloudy', 'Dangerously Windy and Partly Cloudy', 'Dry', 'Windy', 'Humid and Overcast', 'Light Rain', 'Drizzle', 'Windy and Dry', 'Dry and Mostly Cloudy', 'Breezy and Dry'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='precipitation', vocabulary_list=('rain', 'snow', ''), dtype=tf.string, default_value=-1, num_oov_buckets=0), NumericColumn(key='Temperature (C)', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Apparent Temperature (C)', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(x_train, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(x_test,y_test, num_epochs=1, shuffle=False)



In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
# We create a linear estimtor by passing the feature columns we created earlier
linear_est.train(train_input_fn)  # train
result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on tetsing data

clear_output()  # clears console output
print(result['accuracy'])  # the result variable is simply a dict of stats about our model

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_nuretwm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/tensorflow_estimator/python/estimator/canned/linear.py:1478: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  getter=tf.compat.v1.get_variable)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp_nuretwm/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...


UnimplementedError: ignored